In [17]:
import os
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from scipy.sparse import csr_matrix

In [18]:
movies = pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')
# 불필요한 컬럼은 지우기
movies = movies.drop(['genres'], axis=1)
ratings=ratings.drop(['timestamp'], axis=1)

In [21]:
from scipy.sparse import csr_matrix
# pivot ratings into movie features -- movie-user 로 이루어진 매트릭스 만들기
movie_user_matrix = ratings.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)


In [27]:
print(movie_user_matrix)

userId   1    2    3    4    5    6    7    8    9    10   ...  662  663  664  \
movieId                                                    ...                  
1        0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0  4.0  0.0  ...  0.0  4.0  3.5   
2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  5.0  0.0  0.0   
3        0.0  0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
5        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
161944   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
162376   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
162542   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
162672   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
163949   0.0  0.0  0.0  0.0 

In [22]:
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [23]:
movie_to_idx = {}
for i in range(len(movies)):
    movie_to_idx[movies.title[i]] =movies['movieId'][i]
print('movie_to_idx :', movie_to_idx)

movie_to_idx : {'Toy Story (1995)': 1, 'Jumanji (1995)': 2, 'Grumpier Old Men (1995)': 3, 'Waiting to Exhale (1995)': 4, 'Father of the Bride Part II (1995)': 5, 'Heat (1995)': 6, 'Sabrina (1995)': 7, 'Tom and Huck (1995)': 8, 'Sudden Death (1995)': 9, 'GoldenEye (1995)': 10, 'American President, The (1995)': 11, 'Dracula: Dead and Loving It (1995)': 12, 'Balto (1995)': 13, 'Nixon (1995)': 14, 'Cutthroat Island (1995)': 15, 'Casino (1995)': 16, 'Sense and Sensibility (1995)': 17, 'Four Rooms (1995)': 18, 'Ace Ventura: When Nature Calls (1995)': 19, 'Money Train (1995)': 20, 'Get Shorty (1995)': 21, 'Copycat (1995)': 22, 'Assassins (1995)': 23, 'Powder (1995)': 24, 'Leaving Las Vegas (1995)': 25, 'Othello (1995)': 26, 'Now and Then (1995)': 27, 'Persuasion (1995)': 28, 'City of Lost Children, The (Cité des enfants perdus, La) (1995)': 29, 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)': 30, 'Dangerous Minds (1995)': 31, 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)': 32, 'Babe (1995

In [24]:
movie_user_mat_sparse = csr_matrix(movie_user_matrix.values)

In [26]:
print(movie_user_mat_sparse)

  (0, 6)	3.0
  (0, 8)	4.0
  (0, 12)	5.0
  (0, 14)	2.0
  (0, 18)	3.0
  (0, 19)	3.5
  (0, 22)	3.0
  (0, 25)	5.0
  (0, 29)	4.0
  (0, 36)	4.0
  (0, 42)	4.0
  (0, 43)	4.0
  (0, 46)	5.0
  (0, 47)	4.0
  (0, 54)	3.0
  (0, 55)	4.0
  (0, 62)	5.0
  (0, 66)	3.0
  (0, 67)	4.0
  (0, 68)	5.0
  (0, 69)	5.0
  (0, 71)	3.5
  (0, 72)	5.0
  (0, 74)	3.0
  (0, 76)	4.0
  :	:
  (9044, 14)	4.0
  (9044, 72)	3.5
  (9045, 14)	0.5
  (9046, 14)	1.0
  (9047, 14)	2.5
  (9048, 61)	4.0
  (9048, 286)	4.5
  (9049, 623)	1.5
  (9050, 14)	1.0
  (9050, 286)	4.0
  (9051, 14)	2.0
  (9052, 14)	4.0
  (9053, 55)	5.0
  (9054, 546)	3.5
  (9055, 546)	4.0
  (9056, 546)	2.5
  (9057, 14)	0.5
  (9058, 72)	3.0
  (9059, 623)	1.0
  (9060, 623)	1.5
  (9061, 286)	5.0
  (9062, 72)	4.5
  (9063, 610)	5.0
  (9064, 610)	3.0
  (9065, 546)	5.0


In [ ]:
df = pd.DataFrame(data = [[]])


In [28]:
import numpy as np
import pandas as pd
df = pd.DataFrame(data = [[2093, 129], [2093, 129], [2094, 129], [2095, 129]], index = [4109, 6633, 6634, 6635], columns = ['instructor_id', 'primary_department_id'])

from scipy.sparse import lil_matrix
sparse_matrix = lil_matrix((df.index.max()+1, len(df.columns)))
for k, column_name in enumerate(df.columns):
    sparse_matrix[df.index.values, np.full(len(df), k)] = df[column_name].values

In [29]:
sparse_matrix = sparse_matrix.tocsc()

In [30]:
print(sparse_matrix)

  (4109, 0)	2093.0
  (6633, 0)	2093.0
  (6634, 0)	2094.0
  (6635, 0)	2095.0
  (4109, 1)	129.0
  (6633, 1)	129.0
  (6634, 1)	129.0
  (6635, 1)	129.0
